In [1]:
# Import necessary libraries
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import gradio as gr

ModuleNotFoundError: No module named 'gradio'

In [2]:
# Load the original dataset (with text data for diseases and symptoms)
df = pd.read_csv('synthetic_detailed_ayurveda_dataset.csv')

# Define X (symptoms/condition) and y (disease)
X = df['Symptoms/Condition']
y = df['Disease']

In [3]:
# Initialize LabelEncoder for the target variable (diseases)
le = LabelEncoder()

# Encode the target variable (diseases)
y_encoded = le.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer for the symptoms
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit the TF-IDF vectorizer on training data and transform both train and test data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [4]:
# Create and train the XGBoost model
ml_model = XGBClassifier(learning_rate=0.05, n_estimators=100,
                         max_depth=4, subsample=0.9,
                         colsample_bytree=0.1, gamma=1,
                         random_state=42)

ml_model.fit(X_train_tfidf, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.1, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [5]:
# Save the trained model
with open('saved_model.pkl', 'wb') as file:
    pickle.dump(ml_model, file)

# Load the saved model
with open('saved_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [7]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.5 MB/s eta 0:00:00


In [9]:
import gradio as gr
# Function to predict disease and map back to original name
def predict_disease(user_input):
    # Transform the user input using the trained TF-IDF vectorizer
    user_input_tfidf = tfidf_vectorizer.transform([user_input])

    # Predict the encoded disease using the trained model
    predicted_class = loaded_model.predict(user_input_tfidf)

    # Map the encoded prediction back to the actual disease name
    predicted_disease = le.inverse_transform(predicted_class)

    return f"Predicted Disease: {predicted_disease[0]}"

# Gradio interface setup
interface = gr.Interface(
    fn=predict_disease,
    inputs=gr.Textbox(label="Enter symptoms separated by commas"),
    outputs="text",
    title="AyurGenixAI",
    description="Enter your symptoms, and the model will predict the corresponding disease."
)

# Launch Gradio app
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e5fde951ee1c3b64b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
